## Homework

In [4]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-02-05 01:23:42--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 64.233.165.194
Connecting to docs.google.com (docs.google.com)|64.233.165.194|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4c4vliretjmjen1v2is3ppsbjrduf4ad/1612477350000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2021-02-05 01:23:44--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4c4vliretjmjen1v2is3ppsbjrduf4ad/1612477350000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 64.233.161.132
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)|64.233.161.132|:443... connected.


In [56]:
import numpy as np
import pandas as pd
from sklearn.impute import *
from scipy.stats import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.feature_selection import *
from sklearn.datasets import *
from sklearn.model_selection import *
%matplotlib inline
from matplotlib import pyplot as plt 
from sklearn.metrics import *
from sklearn.neighbors import *
from sklearn.decomposition import *
from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import make_blobs, make_circles, make_classification, load_iris, load_digits
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn import *
from matplotlib.colors import *
from collections import *
from sklearn.impute import *

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [57]:
import pandas as pd

In [58]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [59]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [60]:
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]


In [61]:
df_id_time = df[drop_columns]
df.drop(drop_columns, axis=1, inplace=True)

In [62]:
df = pd.get_dummies(df, columns=cat_columns)
df.head()

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878,0.023464,...,0,0,0,0,0,0,0,0,1,0
1,64,64.0,16.0,NaN,NaN,NaN,NaN,6.677245e+07,9553,0.336177,...,0,0,0,0,0,0,0,1,0,0
2,83,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507,0.297166,...,0,0,0,0,0,0,1,0,0,0
3,71,49.0,2.0,NaN,NaN,NaN,NaN,4.708040e+06,38075,0.271702,...,0,0,0,0,0,0,0,1,0,0
4,60,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010,0.389354,...,0,0,0,0,0,0,1,0,0,0


In [63]:
df.isna().sum() 

full_sq                    0
life_sq                 4103
floor                    113
max_floor               6303
build_year              8905
                        ... 
ecology_excellent          0
ecology_good               0
ecology_no data            0
ecology_poor               0
ecology_satisfactory       0
Length: 1065, dtype: int64

In [64]:
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df = pd.DataFrame(data=si.fit_transform(df), columns=df.columns)
df.head()

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878.0,0.023464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,64.0,64.0,16.0,17.0,2014.0,2.0,1.0,6.677245e+07,9553.0,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83.0,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507.0,0.297166,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,71.0,49.0,2.0,17.0,2014.0,2.0,1.0,4.708040e+06,38075.0,0.271702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.0,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010.0,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [65]:
df.isna().sum().sum()

0

In [66]:
df_train = df.iloc[:int(df.shape[0] * 0.8), :]
df_test = df.iloc[int(df.shape[0] * 0.8):, :]
display(df.shape)
display(df_train.shape)
display(df_test.shape)
df.head()

(20000, 1065)

(16000, 1065)

(4000, 1065)

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878.0,0.023464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,64.0,64.0,16.0,17.0,2014.0,2.0,1.0,6.677245e+07,9553.0,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83.0,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507.0,0.297166,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,71.0,49.0,2.0,17.0,2014.0,2.0,1.0,4.708040e+06,38075.0,0.271702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.0,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010.0,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [67]:
y_train = df_train['price']
X_train = df_train.drop(['price'], axis=1)
y_test = df_test['price']
X_test = df_test.drop(['price'], axis=1)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [69]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

DecisionTreeRegressor()

Проверьте качество на отложенной выборке.

In [70]:
y_pred = dtr.predict(X_test)
display('Mean absolute error: ')
display(mean_absolute_error(y_test, y_pred))
display('R2 error')
display(r2_score(y_test, y_pred))

'Mean absolute error: '

1970821.2180833335

'R2 error'

0.3997540777291323

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [73]:
df_train

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878.0,0.023464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,64.0,64.0,16.0,17.0,2014.0,2.0,1.0,6.677245e+07,9553.0,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83.0,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507.0,0.297166,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,71.0,49.0,2.0,17.0,2014.0,2.0,1.0,4.708040e+06,38075.0,0.271702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60.0,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010.0,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,46.0,29.0,3.0,5.0,1965.0,2.0,5.0,9.946335e+06,139322.0,0.065409,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15996,72.0,43.0,16.0,17.0,1995.0,3.0,10.0,8.287309e+06,125354.0,0.071120,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15997,38.0,22.0,8.0,12.0,1968.0,2.0,6.0,1.509487e+07,219609.0,0.300284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15998,67.0,43.0,16.0,16.0,1976.0,3.0,7.0,1.428699e+07,157010.0,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [74]:
month_year = (df_id_time.timestamp.dt.month + df_id_time.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df_id_time.timestamp.dt.weekofyear + df_id_time.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [75]:
df['month'] = df_id_time.timestamp.dt.month
df['weekday'] = df_id_time.timestamp.dt.weekday
df['perc_floor'] = (df.floor / df.max_floor).apply(float)
df['perc_kitchen'] = (df.kitch_sq / df.full_sq).apply(float)
display(df[['month', 'weekday', 'perc_floor', 'perc_kitchen']])

,month,weekday,perc_floor,perc_kitchen
0,12,4,1.000000,1.000000
1,10,3,0.941176,0.015625
2,2,2,0.529412,0.120482
3,7,3,0.117647,0.014085
4,10,2,1.000000,0.100000
...,...,...,...,...
19995,9,0,0.222222,0.111111
19996,12,4,0.235294,0.026316
19997,6,3,0.409091,0.285714
19998,1,1,0.470588,0.176471


In [76]:
df = df.replace([np.inf, -np.inf], np.nan)

In [77]:
df.isna().sum().sum()

343

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [78]:
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df = pd.DataFrame(data=si.fit_transform(df), columns=df.columns)
df.head()

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory,month_year_cnt,week_year_cnt,month,weekday,perc_floor,perc_kitchen
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878.0,0.023464,...,0.0,0.0,1.0,0.0,1122.0,112.0,12.0,4.0,1.000000,1.000000
1,64.0,64.0,16.0,17.0,2014.0,2.0,1.0,6.677245e+07,9553.0,0.336177,...,0.0,1.0,0.0,0.0,327.0,68.0,10.0,3.0,0.941176,0.015625
2,83.0,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507.0,0.297166,...,1.0,0.0,0.0,0.0,752.0,192.0,2.0,2.0,0.529412,0.120482
3,71.0,49.0,2.0,17.0,2014.0,2.0,1.0,4.708040e+06,38075.0,0.271702,...,0.0,1.0,0.0,0.0,262.0,69.0,7.0,3.0,0.117647,0.014085
4,60.0,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010.0,0.389354,...,1.0,0.0,0.0,0.0,711.0,214.0,10.0,2.0,1.000000,0.100000


In [79]:
df_train = df.iloc[:int(df.shape[0] * 0.8), :]
df_test = df.iloc[int(df.shape[0] * 0.8):, :]
display(df.shape)
display(df_train.shape)
display(df_test.shape)
y_train = df_train['price']
X_train = df_train.drop(['price'], axis=1)
y_test = df_test['price']
X_test = df_test.drop(['price'], axis=1)

(20000, 1071)

(16000, 1071)

(4000, 1071)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [80]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

DecisionTreeRegressor()

In [81]:
y_pred = dtr.predict(X_test)
display('Mean absolute error: ')
display(mean_absolute_error(y_test, y_pred))
display('R2 error')
display(r2_score(y_test, y_pred))

'Mean absolute error: '

2013526.54075

'R2 error'

0.34179550054394026

In [82]:
dir(DecisionTreeRegressor)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_n_features',
 '_compute_partial_dependence_recursion',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_prune_tree',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_X_predict',
 '_validate_data',
 'apply',
 'cost_complexity_pruning_path',
 'decision_path',
 'feature_importances_',
 'fit',
 'get_depth',
 'get_n_leaves',
 'get_params',
 'predict',
 'score',
 'set_params']

** Получилось даже хуже (( Попробуем подобрать хорошие гиперпараметры **

In [83]:
X = df.drop(['price'], axis=1)
y = df['price']
display(X.shape)
display(y.shape)

(20000, 1070)

(20000,)

In [84]:
dtr = DecisionTreeRegressor(min_samples_leaf = 10, max_depth = 60)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
display('Mean absolute error: ')
display(mean_absolute_error(y_test, y_pred))
display('R2 error')
display(r2_score(y_test, y_pred))

'Mean absolute error: '

1813411.6252669888

'R2 error'

0.5372974705894802

In [85]:
gscv = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid={'min_samples_leaf': [17, 20, 23, 25, 27], 'max_depth': [10, 17, 20, 23]}, cv=5, n_jobs=-1)
display(gscv)
grid_result = gscv.fit(X, y)
best_params = grid_result.best_params_
best_params

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 17, 20, 23],
                         'min_samples_leaf': [17, 20, 23, 25, 27]})

{'max_depth': 10, 'min_samples_leaf': 27}

In [86]:
%%time
dtr = DecisionTreeRegressor(min_samples_leaf = best_params['min_samples_leaf'], max_depth = best_params['max_depth'])
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
display('Mean absolute error: ')
display(mean_absolute_error(y_test, y_pred))
display('R2 error')
display(r2_score(y_test, y_pred))

'Mean absolute error: '

1722404.7106588813

'R2 error'

0.5774769520054333

CPU times: user 1.6 s, sys: 96.3 ms, total: 1.7 s
Wall time: 1.7 s


** Получилось неплохо так улучшить результат **

In [87]:
# Посмотрим на не модифицированный random forest
rfr = RandomForestRegressor(n_jobs=-1)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
display('Mean absolute error: ')
display(mean_absolute_error(y_test, y_pred))
display('R2 error')
display(r2_score(y_test, y_pred))

'Mean absolute error: '

1475270.6878675001

'R2 error'

0.6833251790733077

** Даже не модифицированный случайный лес дает лучший результат, чем модифицированное дерево решений **

In [88]:
# Опробуем catboost
from catboost import CatBoostRegressor
cbr = CatBoostRegressor()
cbr.fit(X_train, y_train)
y_pred = cbr.predict(X_test)
display('Mean absolute error: ')
display(mean_absolute_error(y_test, y_pred))
display('R2 error')
display(r2_score(y_test, y_pred))


0	total: 10.6s	remaining: 5.63s
654:	learn: 1795871.1620656	total: 10.7s	remaining: 5.62s
655:	learn: 1795569.1309674	total: 10.7s	remaining: 5.6s
656:	learn: 1794577.3274826	total: 10.7s	remaining: 5.58s
657:	learn: 1793456.7269938	total: 10.7s	remaining: 5.57s
658:	learn: 1793162.7048532	total: 10.7s	remaining: 5.55s
659:	learn: 1792445.3849217	total: 10.7s	remaining: 5.53s
660:	learn: 1791581.7903672	total: 10.8s	remaining: 5.51s
661:	learn: 1791331.3206786	total: 10.8s	remaining: 5.5s
662:	learn: 1790376.5292774	total: 10.8s	remaining: 5.48s
663:	learn: 1790189.2170468	total: 10.8s	remaining: 5.47s
664:	learn: 1789628.0645965	total: 10.8s	remaining: 5.45s
665:	learn: 1788792.9935163	total: 10.8s	remaining: 5.44s
666:	learn: 1788431.8826231	total: 10.9s	remaining: 5.42s
667:	learn: 1787404.5967827	total: 10.9s	remaining: 5.4s
668:	learn: 1786961.6539961	total: 10.9s	remaining: 5.39s
669:	learn: 1786877.2584552	total: 10.9s	remaining: 5.37s
670:	learn: 1785868.9439535	total: 10.9s	re

'Mean absolute error: '

1416262.8764903017

'R2 error'

0.7201532930786071

** Ну catboost явно показал поразительные результаты **

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).